In [1]:
import pandas as pd

df = pd.read_excel('data/companies.xlsx')

df

,company,cvr,yearly result latest (in thousands),yearly result previous (in thousands),yearly result before (in thousands),employees
0,NorthTech ApS,10121361,5.0,598.0,792.0,20
1,PFA,13594376,-79000.0,108000.0,10000.0,1464
2,Netcompany A/S,14814833,583573.0,570944.0,450133.0,2340
3,Elbek-vejrup,20117982,43935.0,54376.0,44403.0,171
4,ELLAB A/S,20896949,136500.0,98092.0,90317.0,166
5,Wilke A/S,21830445,3834.0,2006.0,1561.0,175
6,Novo Nordisk,24256790,47523000.0,42159000.0,39252000.0,22150
7,KMD A/S,26911745,53926.0,2893.0,173758.0,1855
8,Alpha Solutions,27506488,3477.0,4334.0,-2940.0,59
9,Formpipe,29177015,36583.0,14719.0,7525.0,87
